In [3]:
import pandas as pd
my_books=pd.read_csv('Downloads/liked_books.csv', index_col=0)
my_books.head()

,user_id,book_id,rating,title
0,-1,2517439,5,"The Forever War (The Forever War, #1)"
1,-1,113576,5,The Smartest Guys in the Room: The Amazing Ris...
2,-1,35100,5,Battle Cry of Freedom
3,-1,228221,5,The Mask of Command
5,-1,17662739,5,"2001: A Space Odyssey (Space Odyssey, #1)"


In [4]:
my_books['book_id']=my_books['book_id'].astype(str)

In [6]:
csv_book_mapping={}
with open('Downloads/book_id_map.csv', 'r') as f:
    while True:
        line=f.readline()
        if not line:
            break
        csv_id, book_id=line.strip().split(",")
        csv_book_mapping[csv_id]=book_id


In [7]:
book_set=set(my_books['book_id'])

In [9]:
overlap_users={}

with open('Downloads/goodreads_interactions.csv', 'r') as f:
    while True:
        line=f.readline()
        if not line:
            break
        user_id, csv_id, _, rating,_=line.strip().split(",")

        book_id=csv_book_mapping.get(csv_id)

        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id]=1
            else:
                overlap_users[user_id]+=1

In [12]:
len(overlap_users)

316341

In [34]:
filtered_overlap_users= set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])

In [38]:
len(filtered_overlap_users)

1258

In [15]:
interactions_list=[]

with open('Downloads/goodreads_interactions.csv', 'r') as f:
    while True:
        line=f.readline()
        if not line:
            break
        user_id, csv_id, _, rating,_=line.strip().split(",")
        if user_id in filtered_overlap_users:
            book_id=csv_book_mapping[csv_id]
            interactions_list.append([user_id, book_id, rating])

In [16]:
len(interactions_list)

5638701

In [17]:
interactions=pd.DataFrame(interactions_list, columns=['user_id', 'book_id','rating'])

In [20]:
interactions=pd.concat([my_books[['user_id', 'book_id','rating']],interactions])

In [21]:
interactions

,user_id,book_id,rating
0,-1,2517439,5
1,-1,113576,5
2,-1,35100,5
3,-1,228221,5
5,-1,17662739,5
...,...,...,...
5638696,804100,475178,0
5638697,804100,186074,0
5638698,804100,153008,0
5638699,804100,45107,0


In [22]:
interactions['book_id']= interactions['book_id'].astype(str)
interactions['user_id']= interactions['user_id'].astype(str)
interactions['rating']=pd.to_numeric(interactions['rating'])


In [27]:
len(interactions['user_id'].unique())

1259

In [24]:
interactions['user_index']=interactions['user_id'].astype('category').cat.codes

In [25]:
interactions['user_index'].unique()

array([   0,  555, 1216, ..., 1054, 1143, 1183], dtype=int16)

In [26]:
interactions['book_index']=interactions['book_id'].astype('category').cat.codes

In [28]:
len(interactions['book_id'].unique())

802870

In [31]:
from scipy.sparse import coo_matrix

rating_mat_coo= coo_matrix((interactions['rating'], (interactions['user_index'], interactions['book_index'])))

In [32]:
rating_mat_coo

<1259x802870 sparse matrix of type '<class 'numpy.int64'>'
	with 5638728 stored elements in COOrdinate format>

In [39]:
rating_mat=rating_mat_coo.tocsr()

In [40]:
rating_mat

<1259x802870 sparse matrix of type '<class 'numpy.int64'>'
	with 5638728 stored elements in Compressed Sparse Row format>

In [41]:
interactions[interactions['user_id']=='-1']

,user_id,book_id,rating,user_index,book_index
0,-1,2517439,5,0,414880
1,-1,113576,5,0,38971
2,-1,35100,5,0,575858
3,-1,228221,5,0,356004
5,-1,17662739,5,0,214285
6,-1,356824,5,0,581743
7,-1,12125412,5,0,59763
8,-1,139069,5,0,124430
10,-1,76680,5,0,722098
11,-1,1898,5,0,276178


In [42]:
my_index=0

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(rating_mat[my_index,:], rating_mat).flatten()

In [49]:
similarity[2]

0.06143442518998915

In [50]:
import numpy as np

indices=np.argpartition(similarity, -15)[-15:]

In [51]:
indices

array([1188,  942,  218,  129,  496,  435, 1208,  795, 1213, 1210, 1143,
        321,  294,  862,    0], dtype=int64)

In [55]:
similar_user=interactions[interactions['user_index'].isin(indices)].copy()

In [58]:
similar_user=similar_user[similar_user['user_id']!='-1']

In [59]:
similar_user

,user_id,book_id,rating,user_index,book_index
45312,4133,5359,3,942,632143
45313,4133,10464963,4,942,13492
45314,4133,3858,3,942,593622
45315,4133,11827808,4,942,51904
45316,4133,7913305,4,942,732465
...,...,...,...,...,...
5638521,712588,32388712,3,1143,543119
5638522,712588,16322,5,1143,183365
5638523,712588,860543,0,1143,759827
5638524,712588,853510,5,1143,756768


In [60]:
book_recs=similar_user.groupby('book_id').rating.agg(['count', 'mean'])
book_recs

,count,mean
book_id,,
1,6,3.833333
100322,1,0.000000
100365,1,0.000000
10046142,1,0.000000
1005,3,0.000000
...,...,...
99561,2,2.500000
99610,1,3.000000
99664,1,4.000000


In [62]:
book_titles=pd.read_json('book_title.json')
book_titles['book_id']= book_titles['book_id'].astype(str)

In [63]:
book_recs=book_recs.merge(book_titles, how='inner', on='book_id')
book_recs

,book_id,count,mean,title,rating_count,url,cover_image,mod_titles
0,1,6,3.833333,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the half blood prince harry p...
1,100322,1,0.000000,Assata: An Autobiography,11057,https://www.goodreads.com/book/show/100322.Assata,https://images.gr-assets.com/books/1328857268m...,assata an autobiography
2,100365,1,0.000000,The Mote in God's Eye,48736,https://www.goodreads.com/book/show/100365.The...,https://images.gr-assets.com/books/1399490037m...,the mote in god s eye
3,10046142,1,0.000000,Dancing in the Glory of Monsters: The Collapse...,2391,https://www.goodreads.com/book/show/10046142-d...,https://images.gr-assets.com/books/1328757755m...,dancing in the glory of monsters the collapse ...
4,1005,3,0.000000,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,think and grow rich
...,...,...,...,...,...,...,...,...
2839,99561,2,2.500000,Looking for Alaska,804587,https://www.goodreads.com/book/show/99561.Look...,https://images.gr-assets.com/books/1394798630m...,looking for alaska
2840,99610,1,3.000000,The Best Laid Plans,17434,https://www.goodreads.com/book/show/99610.The_...,https://images.gr-assets.com/books/1353374848m...,the best laid plans
2841,99664,1,4.000000,The Painted Veil,24606,https://www.goodreads.com/book/show/99664.The_...,https://images.gr-assets.com/books/1320421719m...,the painted veil
2842,9969571,3,2.333333,Ready Player One,376328,https://www.goodreads.com/book/show/9969571-re...,https://images.gr-assets.com/books/1500930947m...,ready player one


In [64]:
book_recs['adjusted_count']=book_recs['count']*(book_recs['count']/book_recs['rating_count'])


In [65]:
book_recs['score']=book_recs['mean']*book_recs['adjusted_count']

In [66]:
book_recs=book_recs[~book_recs['book_id'].isin(my_books['book_id'])]

In [67]:
my_books['mod_title']=my_books['title'].str.replace('[^a-zA-Z0-9 ]', "", regex=True).str.lower()

In [68]:
my_books['mod_title']= my_books['mod_title'].str.replace('\s+', " ", regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Ramadhir\AppData\Local\Temp\ipykernel_14024\1721506036.py:1: SyntaxWarning: invalid escape sequence '\s'
  my_books['mod_title']= my_books['mod_title'].str.replace('\s+', " ", regex=True)


In [70]:
book_recs=book_recs[~book_recs['mod_titles'].isin(my_books['mod_title'])]

In [72]:
book_recs=book_recs[book_recs['count']>2]
book_recs=book_recs[book_recs['mean']>3]

In [73]:
top_recs=book_recs.sort_values('score', ascending=True)

In [76]:
top_recs.shape

(45, 10)

In [82]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

top_recs[['title','url', 'cover_image','score']].style.format({'url': make_clickable, 'cover_image': show_image})

,title,url,cover_image,score
624,"The Notebook (The Notebook, #1)",Goodreads,,0.000031
29,Jane Eyre,Goodreads,,0.000046
601,The Little Prince,Goodreads,,0.000051
958,Gone Girl,Goodreads,,0.000056
1534,"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",Goodreads,,0.000057
1440,"The Hunger Games (The Hunger Games, #1)",Goodreads,,0.000063
2470,Animal Farm,Goodreads,,0.000065
1905,Fahrenheit 451,Goodreads,,0.000066
244,"The Golden Compass (His Dark Materials, #1)",Goodreads,,0.000070
2700,Of Mice and Men,Goodreads,,0.000080
